<a href="https://colab.research.google.com/github/mohamedfadul/ANN-Baseline-Model-to-Predict-Trail-Validity/blob/main/ANN_Baseline_Model_to_Predict_Trail_Validity_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
pip install tensorflow-addons
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow_addons import losses

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

print(tf.__version__)

2.12.0


In [ ]:
all_dataset = pd.read_csv()

In [ ]:
dataset = all_dataset.sample(frac=0.80,random_state=0)
test_dataset = all_dataset.drop(dataset.index)

In [ ]:
dataset['sboxes'] = ((dataset['sboxes']/(dataset['round']*8)))

In [ ]:
test_dataset['sboxes'] = ((test_dataset['sboxes']/(test_dataset['round']*8)))

In [ ]:
# Class count
count_class_0, count_class_1 = dataset.validity.value_counts()

# Divide by class
df_class_0 = dataset[dataset['validity'] == 0]
df_class_1 = dataset[dataset['validity'] == 1]

In [ ]:
df_class_0.shape

In [ ]:
df_class_1.shape

In [ ]:
#UnderSampling
# Undersample 0-class and concat the DataFrames of both class
df_class_0_under = df_class_0.sample(count_class_1)
train_dataset = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(train_dataset.validity.value_counts())

In [ ]:
train_labels = train_dataset.pop("validity")
test_labels = test_dataset.pop("validity")

In [ ]:
train_dataset['round'] = (train_dataset['round'])/100

In [ ]:
test_dataset['round'] = (test_dataset['round'])/100

In [ ]:
from keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def root_mean_squared_error(y_true, y_pred):
        y_pred = ops.convert_to_tensor_v2(y_pred)
        y_true = math_ops.cast(y_true, y_pred.dtype)
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
from keras.layers import Dropout
from keras import regularizers
from keras.regularizers import l2, l1
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

reg_param=0.001
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        layers.Dense(512, activation='relu' , input_shape=[50]),
        layers.Dense(512, activation='relu'),    
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),     
      layers.Dense(1,activation='sigmoid' )
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    early_stop = keras.callbacks.EarlyStopping(monitor='accuracy', patience=10,restore_best_weights=True)
    if weights == -1:
        model.fit(X_train, y_train, epochs=50, callbacks=[early_stop])
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    print("Classification Report: \n", classification_report(y_test, y_preds))
    
    return model

In [ ]:
model = ANN(train_dataset, train_labels, test_dataset, test_labels, tf.keras.losses.BinaryCrossentropy(), -1)

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/Truncated path validity/truncated_validity_model.h5')